In [1]:
import requests
import ast
import json
import pandas as pd
from koeda import EDA
import re

In [42]:
testset = {}

In [56]:
for feedback_id in range(54, 74):
    response = requests.get(f"http://localhost:8000/api/product/feedback_id/{feedback_id}").json()
    recom_list = response["recommendations"]
    best_id = int(response['best'])
    query = response['query']

    response_list = requests.post(f"http://localhost:8000/api/product/prod_id_list/", data=recom_list).json()
    search_name = response_list[0]["search_name"]
    hard_examples = [item["summary"][1:-1] for item in response_list if item['product_id'] != best_id]

    product_list = requests.get(f'http://localhost:8000/api/product/prod_id_list/all/{search_name}').json()

    answer_context = None
    no_answer_context = []
    for product in product_list['product']:
        if product['product_id'] == best_id:
            answer_context = product['summary'][1:-1]
        else:
            if product['summary']:
                no_answer_context.append(product['summary'][1:-1])
    total_contexts = [answer_context] + no_answer_context
    cur_data = {
        "query": query,
        "positive_passage": answer_context,
        "total_passages": total_contexts,
        "hard_negatives": hard_examples
    }
    testset[f'feedback_id_{feedback_id}'] = cur_data

In [58]:
import json

with open('test.json', 'w', encoding='utf-8') as f:
    json.dump(testset, f, ensure_ascii=False, indent='\t')

In [62]:
feedback_id_list = []
query_list = []
positive_passage_list = []
total_passages_list = []
hard_negatives_list = []
for key, val in testset.items():
    feedback_id_list.append(key)
    query_list.append(val['query'])
    positive_passage_list.append(val['positive_passage'])
    total_passages_list.append(val['total_passages'])
    hard_negatives_list.append(val['hard_negatives'])

In [65]:
testset_df = pd.DataFrame({"id":feedback_id_list, 
                           "query": query_list,
                           "positive_context": positive_passage_list,
                           "contexts": total_passages_list,
                           "hard_examples":hard_negatives_list})

In [67]:
testset_df.to_csv('testset.csv', index=False)

In [8]:
response = requests.get(f"http://localhost:8000/api/product/feedback_id/all").json()

In [10]:
feedbacks = response['data']

In [38]:
for idx, feedback in enumerate(feedbacks):
    if feedback['feedback_id'] == 77:
        print(idx)

28


In [ ]:
for response in feedbacks:
    feedback_id = response['feedback_id']
    recom_list = response["recommendations"]
    best_id = int(response['best'])
    query = response['query']

    response_list = requests.post(f"http://localhost:8000/api/product/prod_id_list/", data=recom_list).json()
    search_name = response_list[0]["search_name"]
    hard_examples = [item["summary"][1:-1] for item in response_list if item['product_id'] != best_id]

    product_list = requests.get(f'http://localhost:8000/api/product/prod_id_list/all/{search_name}').json()

    answer_context = None
    no_answer_context = []
    for product in product_list['product']:
        if product['product_id'] == best_id:
            answer_context = product['summary'][1:-1]
        else:
            if product['summary']:
                no_answer_context.append(product['summary'][1:-1])
    total_contexts = [answer_context] + no_answer_context
    cur_data = {
        "query": query,
        "positive_passage": answer_context,
        "total_passages": total_contexts,
        "hard_negatives": hard_examples
    }
    testset[f'feedback_id_{feedback_id}'] = cur_data

In [48]:
len(testset)

40

In [49]:
feedback_id_list = []
query_list = []
positive_passage_list = []
total_passages_list = []
hard_negatives_list = []
for key, val in testset.items():
    feedback_id_list.append(key)
    query_list.append(val['query'])
    positive_passage_list.append(val['positive_passage'])
    total_passages_list.append(val['total_passages'])
    hard_negatives_list.append(val['hard_negatives'])

In [50]:
import pandas as pd

testset_df = pd.DataFrame({"id":feedback_id_list, 
                           "query": query_list,
                           "positive_context": positive_passage_list,
                           "contexts": total_passages_list,
                           "hard_examples":hard_negatives_list})
testset_df.to_csv('testset_40.csv', index=False)

In [51]:
with open('test_40.json', 'w', encoding='utf-8') as f:
    json.dump(testset, f, ensure_ascii=False, indent='\t')

In [53]:
one_hard = [hard[0] for hard in hard_negatives_list]

In [55]:
hard_df = pd.DataFrame({"id":feedback_id_list, "question": query_list, "context":positive_passage_list, "negative_example":one_hard})

In [57]:
hard_df.head(3)

,id,question,context,negative_example
0,feedback_id_36,맵고 식감이 쫄깃하고 양 많은 것,<맛> 떡이 진짜 맛있고 양념도 기가 막히게 맛있어서 <양> 가격도 저렴하고 양도 ...,<맛> 최고 <선호> 아이들이 매운 거 못 먹어서 <양> 4인분이라서 2인분씩 먹으...
1,feedback_id_42,바삭바삭하고 닭가슴살,<맛> 맛있게 먹으려고 여러 노력 <양념치킨 소스> 너무 맛있어서 또 구매 <다이어...,<맛> 맵지 않고 맛있어요 <양> 양도 많아서 당분간 걱정 없이 먹겠네요 <선호> ...
2,feedback_id_47,비계 적은 것,<맛> 기름이 많이 있지 않아서 제일 마음에 듬 <양> 다이어트 식을 만들기에도 좋...,<맛> 고기 색깔만 봐도 신선함이 느껴져요 <양> 1팩 500g <조리> 냉동상태로...


In [66]:
hard_df = hard_df.sample(frac=1)

In [69]:
hard_df.to_csv('hard_data.csv', index=False)

In [73]:
hard_df.iloc[35:].to_csv('hard_val.csv', index=False)

## 요약 전체 가져오기

In [2]:
response = requests.get(f"http://localhost:8000/api/product/summary/all").json()

In [207]:
product_ids = [res['product_id'] for res in response['data']]

In [4]:
summary_list = [res['summary'][1:-1] for res in response['data'] if len(res['summary']) <= 200]

In [11]:
eda = EDA(
    morpheme_analyzer="Okt", alpha_sr=1.0, alpha_ri=0.0, alpha_rs=0.0, prob_rd=0.0
)

In [222]:
def remove_tags_eda(text):
    pattern = r'<[^>]+>'
    result = re.sub(pattern, '|', text)
    result_list = result.split('| ')
    return [eda(result.rstrip()) for result in result_list if result.rstrip() and len(result.rstrip()) <= 25]

def remove_tags(text):
    pattern = r'<[^>]+>'
    result = re.sub(pattern, '|', text)
    result_list = result.split('| ')
    return " ".join([result.rstrip() for result in result_list if result.rstrip()])

text = '<맛> 곰곰 우리 쌀이 제일 <가격> 가격에 부담스럽지 않음 <품질> 쌀 품질도 좋음 <향> 배송도 꼼꼼하게 잘 옴 <선호> 밥할 때 구수한 냄새가 입맛을 돋아나게 함'
result_list = remove_tags(text)
# ' '.join(result_list)
print(result_list)

곰곰 우리 쌀이 제일 가격에 부담스럽지 않음 쌀 품질도 좋음 배송도 꼼꼼하게 잘 옴 밥할 때 구수한 냄새가 입맛을 돋아나게 함


In [223]:
ids = []
query = []
context = []
for p_id, summary in zip(product_ids, summary_list):
    query_list = remove_tags_eda(summary)
    context_list = remove_tags(summary)
    query.extend(query_list)
    ids.extend([ids for _ in range(len(query_list))])
    context.extend([context_list for _ in range(len(query_list))])

In [224]:
len(query)

16627

In [229]:
ids = [f'data_id_{id_}' for id_ in list(range(1, len(query)+1))]

In [232]:
train = pd.DataFrame({"id": ids, "query": query, "context":context})

In [233]:
train.head(5)

,id,query,context
0,data_id_1,대등 곰곰 우리 쌀이 제일,곰곰 우리 쌀이 제일 가격만 저렴하고 쌀 품질도 좋음 구수한 냄새가 입맛을 돋아나게...
1,data_id_2,가격만 저렴하고 쌀 품질도 좋음,곰곰 우리 쌀이 제일 가격만 저렴하고 쌀 품질도 좋음 구수한 냄새가 입맛을 돋아나게...
2,data_id_3,구수한 냄새가 입맛을 돋아나게 함,곰곰 우리 쌀이 제일 가격만 저렴하고 쌀 품질도 좋음 구수한 냄새가 입맛을 돋아나게...
3,data_id_4,꼼꼼하게 잘 오고 집 전도까지 가져다주심,곰곰 우리 쌀이 제일 가격만 저렴하고 쌀 품질도 좋음 구수한 냄새가 입맛을 돋아나게...
4,data_id_5,다섯 식구인쌀저희는 비싼 은 못 먹겠음,곰곰 우리 쌀이 제일 가격만 저렴하고 쌀 품질도 좋음 구수한 냄새가 입맛을 돋아나게...
